In [1]:
from enlp import DependencyParser, POS, SentenceTokenizer, NER

In [2]:
from nltk.corpus import wordnet as wn
import numpy as np
import pdb

In [3]:
dependparser = DependencyParser()
pos_tagger = POS()
ner = NER()
sen_tokenizer = SentenceTokenizer()

2019-07-25 00:14:30,566     INFO   59 corenlp:backend.__init__: Executing `java -Xms2g -Xmx3000m -cp /home/tittit/.enlp/data/corenlp/stanford-corenlp-3.8.0.jar:/home/tittit/.enlp/data/corenlp/stanford-corenlp-3.8.0-models.jar edu.stanford.nlp.pipeline.StanfordCoreNLP -props /home/tittit/.enlp/data/corenlp/english.properties`


In [4]:
verb_dictionary = {}
verb_dictionary['verb_base_form'] = []
verb_dictionary['verb_3rd_present'] = []
verb_dictionary['verb_gerund'] = []
verb_dictionary['verb_past_tense'] = []
verb_dictionary['verb_past_participle'] = []
f = open('/home/tittit/challenges/VAGC2019/HN_challenge/ielts/gens/read/paraphrase/verb.tsv', 'r')
for row in f:
    row_split = row[:-1].split('\t')
    verb_dictionary['verb_base_form'].append(row_split[0])
    verb_dictionary['verb_3rd_present'].append(row_split[1])
    verb_dictionary['verb_gerund'].append(row_split[2])
    verb_dictionary['verb_past_tense'].append(row_split[3])
    verb_dictionary['verb_past_participle'].append(row_split[4])

In [5]:
def get_morphy(verb, tag):
	morphy = verb
	if (tag in ['VB', 'VBP']):
		morphy = verb
	if (tag == 'VBZ'):
		if (verb.lower() in verb_dictionary['verb_3rd_present']):
			morphy = verb_dictionary['verb_base_form'][verb_dictionary['verb_3rd_present'].index(verb.lower())]
	if (tag == 'VBD'):
		if (verb.lower() in verb_dictionary['verb_past_tense']):
			morphy = verb_dictionary['verb_base_form'][verb_dictionary['verb_past_tense'].index(verb.lower())]		
	if (tag == 'VBN'):
		if (verb.lower() in verb_dictionary['verb_past_participle']):
			morphy = verb_dictionary['verb_base_form'][verb_dictionary['verb_past_participle'].index(verb.lower())]
	if (tag == 'VBG'):
		if (verb.lower() in verb_dictionary['verb_gerund']):
			morphy = verb_dictionary['verb_base_form'][verb_dictionary['verb_gerund'].index(verb.lower())]
	return morphy

In [6]:
def dfs(i, result, status, relations):
	if (i >= len(relations)):
		status_ = [i for i in status]
		result.append(status_)
		return result

	for x in relations[i]:
		status.append(x)
		result = dfs(i + 1, result, status, relations)
		status.remove(x)

	return result

In [7]:
def get_dp_conll(raw_dp, words, xpos, ners):
    # declare list of dependency trees
    dependency_trees = []

    # make a tuple of relations
    relations = []

    for dependency in raw_dp['dependparser'][0]:
        head = dependency.root_index + 1
        tail = dependency.target_index + 1
        label = dependency.label
        if (label == 'nsubj:xsubj'):
            continue
        is_exist = False
        for i in range(len(relations)):
            for j in range(len(relations[i])):
                if ((relations[i][j][0] == head) and (relations[i][j][1] == tail)) or ((relations[i][j][0] == tail) and (relations[i][j][1] == head)):
                    is_exist = True
                    relations[i].append([head, tail, label])
                    break
            if is_exist:
                break
        if not is_exist:
            relations.append([[head, tail, label]])
    # devide all relation in to corespnding no-loop trees
    dependency_relations = dfs(0, [], [], relations)

    # generate dependency trees
    for dependency_relation in dependency_relations:
        # declare lists
        labels = ['']
        heads = [-1]
        for i in range(1, len(words)):
            labels.append('root')
            heads.append(0)
        # generate tree
        for relation in dependency_relation:
            if (heads[relation[1]] != 0) and (labels[relation[1]] == 'conj:and') and (relation[2] in ['nsubj', 'ccomp', 'xcomp']):
                continue
            heads[relation[1]] = relation[0]
            labels[relation[1]] = relation[2]
        count_root = 0
        for head in heads:
            if head == 0:
                count_root += 1
        if count_root > 1:
            continue
        dependency_trees.append({'words' : words, 'xpos' : xpos, 'heads' : heads, 'labels': labels, 'ners' : ners})

    return dependency_trees

In [8]:
def get_pos_conll(raw_pos):
	words = ['']
	xpos = ['']

	for pos in raw_pos[0]:
		words.append(pos.word)
		xpos.append(pos.pos_tag)

	return words, xpos

In [9]:
def get_ner_conll(raw_ner):
	ners = ['']
	for ner in raw_ner[0]:
		ners.append(ner.ner_tag)
	return ners

In [10]:
def check_belong_to(i, j, heads):
	cnt = 0
	while (j != 0) and (j != i):
		j = heads[j]
		cnt += 1
		if (cnt == 100):
			return False
	if (j == i):
		return True
	else:
		return False

In [11]:
def get_all_word_belong_to(i, words, heads):
	answer = ''
	last_answer_id = -1
	for j in range(1, len(heads)):
		if check_belong_to(i, j, heads):
			answer += ' ' + words[j]
			last_answer_id = j
	return answer, last_answer_id

In [12]:
def get_noun_info(noun, tag):
	noun_labels = ['NN', 'NNP', 'NNS', 'NNPS']
	personal_pronoun_labels = ['PRP']
	singular_personal_pronoun = ['i', 'she', 'he', 'it']
	plural_personal_pronoun = ['we', 'you', 'they']
	### noun type is singular (0) or prural (1)
	noun_type = 0
	is_human = False
	if (tag in noun_labels):
		if (tag in ['NNS', 'NNPS']):
			noun_type = 1

		### noun points to Human or not
		synsets = wn.synsets(str(wn.morphy(noun.lower())), 'n')
		if (len(synsets) != 0):
			cnt = 0
			noun = synsets[0]
			while (len(noun.hypernyms()) != 0) and (cnt < 100):
				cnt += 1
				if (noun.name()[:6] == 'person'):
					is_human = True
					break
				noun = noun.hypernyms()[0]
	if (tag in personal_pronoun_labels + plural_personal_pronoun):
		if (noun.lower() in plural_personal_pronoun):
			noun_type = 1
		if (noun.lower() in ['i', 'we', 'she', 'he', 'you', 'they']):
			is_human = True

	### return
	return is_human, noun_type

In [13]:
"""
    dobj: direct object
    70%: VERB -- dobj -- NOUN
    Eg: She gave me a raise
    question: what she gave me? = what + clausal (all words belong to Verb except dobj part which will be answer)
    answer  : a raise = all words belong to NOUN

    error: 1 câu hoặc 1 mệnh đề có thể có nhiều dobj
"""

def generate_dobj_question(words, xpos, heads, labels, ners):
    ###
    questions = []
    answers = []
    relations = []
    verb_labels = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    adj_labels = ['JJ', 'JJR', 'JJS']
    noun_labels = ['NN', 'NNP', 'NNS', 'NNPS', 'PRP']
    nsubj_labels = ['nsubj', 'nsubjpass']
    ### find all dobj relations
    for i in range(1, len(labels)):
        if (labels[i] == 'dobj'):
            head_id = heads[i]
            tail_id = i
            ### VERB ----dobj----> NOUN
            if ((xpos[head_id] in verb_labels) and (xpos[tail_id] in noun_labels)):
                # get nsubj of sentence
                nsubj = ''
                for j in range(1, len(labels)):
                    if ((heads[j] == head_id) and (labels[j] in nsubj_labels)):
                        nsubj = ''
                        illegal_id = []
                        if xpos[j] in ['NNP', 'NNPS']:
                            for k1 in range(1, len(heads)):
                                if (heads[k1] == j) and ((labels[k1] in ['appos', 'punct']) or ((len(labels[k1]) > 4) and (labels[k1][:4] == 'nmod'))):
                                    for k2 in range(1, len(heads)):
                                        if check_belong_to(k1, k2, heads):
                                            illegal_id.append(k2)
                        for k in range(1, len(labels)):
                            if (k not in illegal_id) and (check_belong_to(j, k, heads)):
                                nsubj += ' ' + words[k]
                        nsubj = nsubj[1:]
                # can not generate question without subject
                if (nsubj == ''):
                    continue

                # get dobj
                dobj = ''
                illegal_id = []
                for j in range(1, len(labels)):
                    if (check_belong_to(i, j, heads)):
                        dobj += ' ' + words[j]
                        illegal_id.append(j)
                dobj = dobj[1:]

                # get dobj info
                is_human, type_noun = get_noun_info(words[tail_id], xpos[tail_id])
                if (ners[tail_id] == 'PERSON'):
                    is_human = True
                for j in range(1, len(heads)):
                    if (heads[j] == tail_id) and (labels[j] in ['conj']):
                        noun_type = 1
                        
                # get body of question: the whole part belong to head except dobj will be the 
                # get aux
                aux_id = -1
                for j in range(1, len(heads)):
                    if (check_belong_to(head_id, j, heads) and (labels[j] in ['aux', 'auxpass'])):
                        aux_id = j
                        break
                        
                # get extension
                extension = ''
                for j in range(head_id, len(heads)):
                    if (heads[j] == head_id) and ((labels[j] in ['advmod']) or ((len(labels[j]) >= 4) and (labels[j][:4] == 'nmod'))):
                        for k in range(head_id, len(heads)):
                            if (check_belong_to(j, k, heads)):
                                extension += ' ' + words[k]
                extension = extension[1:]

                # generate question
                if (is_human):
                    if (aux_id != -1):
                        question = 'who ' + words[aux_id] + ' ' + nsubj + ' ' + words[head_id] + ' ' + extension
                    else:
                        question = 'who ' + nsubj + ' ' + words[head_id] + ' ' + extension
                    if (xpos[head_id] == 'VBZ'):
                        question = 'who does ' + nsubj + ' ' + get_morphy(words[head_id], 'VBZ', verb_dictionary) + ' ' + extension
                    if (xpos[head_id] == 'VBP'):
                        question = 'who do ' + nsubj + ' ' + get_morphy(words[head_id], 'VBP', verb_dictionary) + ' ' + extension
                    if (xpos[head_id] == 'VBD') and (aux_id == -1):
                        question = 'who did ' + nsubj + ' ' + get_morphy(words[head_id], 'VBD', verb_dictionary) + ' ' + extension
                else:
                    if (ners[tail_id] == 'MONEY'):
                        if (aux_id != -1):
                            question = 'How much ' + words[aux_id] + ' ' + nsubj + ' ' + words[head_id] + ' ' + extension
                        else:
                            question = 'How much ' + nsubj + ' ' + words[head_id] + ' ' + extension
                        if (xpos[head_id] == 'VBZ'):
                            question = 'How much does ' + nsubj + ' ' + get_morphy(words[head_id], 'VBZ', verb_dictionary) + ' ' + extension
                        if (xpos[head_id] == 'VBP'):
                            question = 'How much do ' + nsubj + ' ' + get_morphy(words[head_id], 'VBP', verb_dictionary) + ' ' + extension
                        if (xpos[head_id] == 'VBD') and (aux_id == -1):
                            question = 'How much did ' + nsubj + ' ' + get_morphy(words[head_id], 'VBD', verb_dictionary) + ' ' + extension
                    else:
                        if (aux_id != -1):
                            question = 'what ' + words[aux_id] + ' ' + nsubj + ' ' + words[head_id] + ' ' + extension
                        else:
                            question = 'what ' + nsubj + ' ' + words[head_id] + ' ' + extension
                        if (xpos[head_id] == 'VBZ'):
                            question = 'what does ' + nsubj + ' ' + get_morphy(words[head_id], 'VBZ', verb_dictionary) + ' ' + extension
                        if (xpos[head_id] == 'VBP'):
                            question = 'what do ' + nsubj + ' ' + get_morphy(words[head_id], 'VBP', verb_dictionary) + ' ' + extension
                        if (xpos[head_id] == 'VBD') and (aux_id == -1):
                            question = 'what did ' + nsubj + ' ' + get_morphy(words[head_id], 'VBD', verb_dictionary) + ' ' + extension
                answer = dobj
                # append
                questions.append(question + '?')
                answers.append(answer)
                relations.append('dobj')
    ### return
    return questions, answers, relations

In [14]:
"""
    nummod : numeric modifier of a noun
    64% Noun  -- nummod -- Num
    17% PropN -- nummod -- Num
    10% Sym   -- nummod -- Num
"""

def generate_nummod_question(words, xpos, heads, labels, ners):
    ###
    questions = []
    answers = []
    relations = []
    verb_labels = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    adj_labels = ['JJ', 'JJR', 'JJS']
    noun_labels = ['NN', 'NNP', 'NNS', 'NNPS', 'PRP']
    nsubj_labels = ['nsubj', 'nsubjpass']
    ### find all dobj relations
    for i in range(1, len(labels)):
        """
        =========================================================================================
        ==== process sentences have form: subject(CD --nummod-- noun) + verb + ...===============
        =========================================================================================
        """
        if (labels[i] == 'nummod') and (labels[heads[i]] in nsubj_labels) and (xpos[heads[heads[i]]] in verb_labels):
            nummod_id = i
            subj_id = heads[i]
            verb_id = heads[heads[i]]
            ###=== get nummod ===================================================================
            nummod = ''
            for j in range(len(labels)):
                if check_belong_to(nummod_id, j, heads):
                    nummod += ' ' + words[j]
            nummod = nummod[1:]
            ###=== get subject and its compound =================================================
            noun = ''
            for j in range(1, len(heads)):
                if (j == subj_id) or ((heads[j] == subj_id) and (labels[j] in ['compound', 'amod'])):
                    noun += ' ' + words[j]
            noun = noun[1:]
            ###=== generate question ============================================================
            question_body = ''
            for j in range(1, len(heads)):
                if (not check_belong_to(subj_id, j, heads)) and (check_belong_to(heads[subj_id], j, heads)):
                    if (words[j] == 'am'):
                        question_body += ' is'
                    else:
                        question_body += ' ' + words[j]
            if (question_body == ''):
                continue
            question = 'How many ' + noun + ' ' + question_body
            ###=== append =======================================================================
            questions.append(question + '?')
            answers.append(nummod + ' ' + noun)
            relations.append('nummod')

        """
        =========================================================================================
        === process sentences have form: subject + verb + direct_object(CD -- nummod -- noun) ===
        =========================================================================================
        """
        if (labels[i] == 'nummod') and (labels[heads[i]] == 'dobj') and (xpos[heads[heads[i]]] in verb_labels):
            obj_id = heads[i]
            nummod_id = i
            verb_id = heads[obj_id]
            ###=== check if exist nsubj point to head ==========================================
            nsubj_id = -1
            for j in range(1, len(labels)):
                if (check_belong_to(verb_id, j, heads) and (labels[j] in nsubj_labels)):
                    nsubj_id = j
                    break
            if (nsubj_id == -1):
                continue
            ###=== get dobj info ================================================================
            is_human, type_noun = get_noun_info(words[obj_id], xpos[obj_id])
            if (ners[obj_id] == 'PERSON'):
                is_human = True
            for j in range(1, len(heads)):
                if (heads[j] == obj_id) and (labels[j] in ['conj']):
                    noun_type = 1       
            ###=== get body of question: the whole part belong to head except dobj ==============
            ###=== get nsubject =================================================================
            nsubj = ''
            for j in range(1, len(heads)):
                if (check_belong_to(nsubj_id, j, heads)):
                    nsubj += ' ' + words[j]
            nsubj = nsubj[1:]
            ###=== get object and its compound =================================================
            noun = ''
            for j in range(1, len(heads)):
                if (j == obj_id) or ((heads[j] == obj_id) and (labels[j] in ['compound', 'amod'])):
                    noun += ' ' + words[j]
            noun = noun[1:]
            ###=== get nummod ===================================================================
            nummod = ''
            for j in range(len(heads)):
                if (check_belong_to(nummod_id, j, heads)):
                    nummod += ' ' + words[j]
            nummod = nummod[1:]
            ###=== get aux ======================================================================
            aux_id = -1
            for j in range(1, len(heads)):
                if (check_belong_to(verb_id, j, heads) and (labels[j] in ['aux', 'auxpass'])):
                    aux_id = j
                    break
            ###=== get verb =====================================================================
            verb = ''
            for j in range(1, len(heads)):
                if (j == verb_id) or ((heads[j] == verb_id) and (j != aux_id) and (labels[j] in ['aux', 'auxpass'])):
                    verb += ' ' + words[j]
            verb = verb[1:]
            ###=== get extension ================================================================
            extension = ''
            for j in range(verb_id, len(heads)):
                if (heads[j] == verb_id) and (((len(labels[j]) >= 5) and (labels[j][:5] == 'advcl')) or ((len(labels[j]) >= 4) and (labels[j][:4] == 'nmod')) or (labels[j] in ['acl'])):
                    for k in range(verb_id, len(heads)):
                        if (check_belong_to(j, k, heads)):
                            extension += ' ' + words[k]
            extension = extension[1:]
            ###=== generate question ============================================================
            if (aux_id != -1):
                question = 'How many ' + noun + ' ' + words[aux_id] + ' ' + nsubj + ' ' + verb + ' ' + extension
            else:
                question = 'How many ' + noun + ' ' + nsubj + ' ' + verb + ' ' + extension
            if (xpos[verb_id] == 'VBZ'):
                question = 'How many ' + noun + ' does ' + nsubj + ' ' + get_morphy(words[verb_id], xpos[verb_id], verb_dictionary) + ' ' + extension
            if (xpos[verb_id] == 'VBP'):
                question = 'How many ' + noun + ' do ' + nsubj + ' ' + get_morphy(words[verb_id], xpos[verb_id], verb_dictionary) + ' ' + extension
            if (xpos[verb_id] == 'VBD') and (aux_id == -1):
                question = 'How many ' + noun + ' did ' + nsubj + ' ' + get_morphy(words[verb_id], xpos[verb_id], verb_dictionary) + ' ' + extension
            answer = nummod
            ###=== append =======================================================================
            questions.append(question + '?')
            answers.append(answer)
            relations.append('nummod')

        """
        =========================================================================================
        === process sentences have form: (CD --nummod-- noun) + (be --cop-- noun) ===============
        =========================================================================================
        """
        if (labels[i] == 'nummod') and (labels[heads[i]] in nsubj_labels) and (xpos[heads[heads[i]]] in noun_labels):
            nummod_id = i
            subj_id = heads[i]
            obj_id = heads[subj_id]
            ###=== check whether exist cop relation =============================================
            cop_id = -1
            for j in range(1, len(heads)):
                if (heads[j] == obj_id) and (labels[j] == 'cop'):
                    cop_id = j
            if (cop_id == -1):
                continue
            ###=== get nummod ===================================================================
            nummod = ''
            for j in range(1, len(heads)):
                if (check_belong_to(nummod_id, j, heads)):
                    nummod += ' ' + words[j]
            nummod = nummod[1:]
            ###=== get subject ==================================================================
            subject = ''
            for j in range(1, len(heads)):
                if (heads[j] == subj_id) and (labels[j] in ['compound', 'amod', 'det']):
                    subject += ' ' + words[j]
            subject = subject[1:]
            ###=== get tobe =====================================================================
            tobe = ''
            for j in range(1, len(heads)):
                if (heads[j] == obj_id) and (labels[j] in ['cop', 'aux', 'auxpass']):
                    tobe += ' ' + words[j]
            tobe = tobe[1:]
            ###=== get dobject ==================================================================
            dobject = ''
            for j in range(1, len(heads)):
                if (j == obj_id):
                    dobject += ' ' + words[j]
                if ((heads[j] == obj_id) and (labels[j] in ['compound', 'amod', 'nummod'])):
                    for k in range(1, len(heads)):
                        if (check_belong_to(j, k, heads)):
                            dobject += ' ' + words[k]
            dobject = dobject[1:]
            ###=== generate question ============================================================
            question = 'how many ' + subject + ' ' + tobe + ' ' + dobject
            answer = nummod
        """
        =========================================================================================
        === process sentences have form: subject + (be --cop-- (CD --nummod-- noun)) ============
        =========================================================================================
        """
        if (labels[i] == 'nummod') and (labels[heads[i]] == 'root'):
            nummod_id = i
            obj_id = heads[i]
            ###=== get subject id ===============================================================
            subj_id = -1
            for j in range(1, len(heads)):
                if (heads[j] == obj_id) and (labels[j] in nsubj_labels):
                    subj_id = j
            if (subj_id == -1):
                continue
            ###=== get subject ==================================================================
            subject = ''
            for j in range(1, len(heads)):
                if (check_belong_to(subj_id, j, heads)):
                    subject += ' ' + words[j]
            subject = subject[1:]
            ###=== get tobe =====================================================================
            tobe = ''
            for j in range(1, len(heads)):
                if (heads[j] == obj_id) and (labels[j] in ['cop', 'aux', 'auxpass']):
                    tobe += ' ' + words[j]
            tobe = tobe[1:]
            ###=== get dobject ==================================================================
            dobject = ''
            for j in range(1, len(heads)):
                if (j == obj_id):
                    dobject += ' ' + words[j]
                if ((heads[j] == obj_id) and (labels[j] in ['compound'])):
                    for k in range(1, len(heads)):
                        if (check_belong_to(j, k, heads)):
                            dobject += ' ' + words[k]
            dobject = dobject[1:]
            ###=== get nummod ===================================================================
            nummod = ''
            for j in range(1, len(heads)):
                if (check_belong_to(nummod_id, j, heads)):
                    nummod += ' ' + words[j]
            nummod = nummod[1:]
            ###=== generate question ============================================================
            question = 'how many ' + dobject + ' ' + tobe + ' ' + subject
            answer = nummod
            ###=== append =======================================================================
            questions.append(question + '?')
            answers.append(answer)
            relations.append('nummod')
        """
        =========================================================================================
        === process sentences have form: ... + (verb --nmod:tmod-- (CD --nummod-- noun))=========
        =========================================================================================
        """
        if (labels[i] == 'nummod') and (labels[heads[i]] in ['nmod:tmod']) and (xpos[heads[heads[i]]] in verb_labels):
            nummod_id = i
            verb_id = heads[heads[i]]
            illegal_id = []
            ###=== get nsubj of sentence ========================================================
            nsubj = ''
            for j in range(1, len(labels)):
                if ((heads[j] == verb_id) and (labels[j] in nsubj_labels)):
                    nsubj = ''
                    illegal_id = []
                    if xpos[j] in ['NNP', 'NNPS']:
                        for k1 in range(1, len(heads)):
                            if (heads[k1] == j) and ((labels[k1] in ['appos', 'punct']) or ((len(labels[k1]) > 4) and (labels[k1][:4] == 'nmod'))):
                                for k2 in range(1, len(heads)):
                                    if check_belong_to(k1, k2, heads):
                                        illegal_id.append(k2)
                    for k in range(1, len(labels)):
                        if (k not in illegal_id) and (check_belong_to(j, k, heads)):
                            nsubj += ' ' + words[k]
                    nsubj = nsubj[1:]
            ###=== get nummod ===================================================================
            nummod = ''
            for j in range(1, len(heads)):
                if (check_belong_to(nummod_id, j, heads)):
                    nummod += ' ' + words[j]
                    illegal_id.append(j)
            nummod = nummod[1:]
            ###=== get noun and its compound ====================================================
            noun = ''
            for j in range(1, len(heads)):
                if (j == heads[i]) or ((heads[j] == heads[i]) and (labels[j] == 'compound')):
                    noun += ' ' + words[j]
                    illegal_id.append(j)
            noun = noun[1:]
            ###=== get auxiliary of verb ========================================================
            aux = ''
            for j in range(1, len(heads)):
                if (heads[j] == verb_id) and (labels[j] == 'aux'):
                    aux = ''
                    for k in range(1, len(labels)):
                        if (check_belong_to(j, k, heads)):
                            aux += ' ' + words[k]
            aux = aux[1:]
            ###=== get predicate ================================================================
            predicate = ''
            for j in range(1, len(heads)):
                if (heads[j] == verb_id) and (labels[j] not in ['punct', 'aux', 'nsubj', 'nsubjpass']) and not ((len(labels[j]) >= 4) and (labels[j][:4] == 'nmod')):
                    for k in range(1, len(heads)):
                        if check_belong_to(j, k, heads):
                            predicate += ' ' + words[k]
            predicate = predicate[1:]
            ###=== generate question ============================================================
            question = ''
            if aux == '':
                if (xpos[verb_id] == 'VBZ'):
                    question = 'how many ' + noun + ' does '
                if (xpos[verb_id] == 'VBP'):
                    question = 'how many ' + noun + ' do '
                if (xpos[verb_id] == 'VBD'):
                    question = 'how many ' + noun + ' did '
                if (xpos[verb_id] == 'VBN'):
                    question = 'how many ' + noun + ' have '
            elif len(aux.split(' ')) == 1:
                question = 'how many ' + noun + ' ' + aux + ' '
            if (question != '') and (nsubj != ''):
                question += nsubj + ' ' + get_morphy(words[verb_id], xpos[verb_id], verb_dictionary) + ' ' + predicate
            else:
                question += get_morphy(words[verb_id], xpos[verb_id], verb_dictionary) + ' ' + predicate
            answer = nummod
            ###=== append =======================================================================
            if question == '':
                continue
            questions.append(question + '?')
            answers.append(answer)
            relations.append('nummod')

    ### return
    return questions, answers, relations

In [20]:

def generate_nmod_question(words, xpos, heads, labels, ners):
    ###
    questions = []
    answers = []
    list_tags = []
    verb_labels = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    adj_labels = ['JJ', 'JJR', 'JJS']
    noun_labels = ['NN', 'NNP', 'NNS', 'NNPS']
    nsubj_labels = ['nsubj', 'nsubjpass']
    nmod_labels = ['nmod:in', 'nmod:on', 'nmod:at', 'nmod:before', 'nmod:after', 'nmod:near']
    """
        nmod:in -> location
    """
    for i in range(1, len(labels)):
        """
            process sentences have form: subject + verb + dobj + in/on + TIME/LOCATION
        """
        tags = []
        if (labels[i] in nmod_labels) and (xpos[heads[i]] in verb_labels) and (ners[i] in ['TIME', 'DATE', 'LOCATION', 'ORGANIZATION', 'COUNTRY']):
            verb_id = heads[i]
            entity_id = i
            ###=== get subject_id ===============================================================
            subj_id = -1
            for j in range(1, len(heads)):
                if (heads[j] == verb_id) and (labels[j] in nsubj_labels):
                    subj_id = j
            if (subj_id != -1):
                ###=== exist auxulary or not ========================================================
                first_aux = -1
                for j in range(1, len(heads)):
                    if (heads[j] == verb_id) and (labels[j] in ['aux', 'auxpass']):
                        first_aux = j
                        break
                ###=== get subject ==================================================================
                subj = ''
                for j in range(1, len(heads)):
                    if (check_belong_to(subj_id, j, heads)):
                        subj += ' ' + words[j]
                subj = subj[1:]
                ###=== get verb =====================================================================
                illegal_id = []
                verb = ''
                for j in range(1, len(heads)):
                    if (heads[j] == verb_id) and (labels[j] not in ['compound', 'compound:prt', 'compound:svc']):
                        for k in range(1, len(heads)):
                            if (check_belong_to(j, k, heads)):
                                illegal_id.append(k)
                for j in range(1, len(heads)):
                    if (j != first_aux) and (j not in illegal_id) and (check_belong_to(verb_id, j, heads)):
                        verb += ' ' + words[j]
                verb = verb[1:]
                ###=== get verb compound=====================================================================
                compound = ''
                for j in range(1, len(heads)):
                    if (heads[j] == verb_id) and (labels[j] in ['compound:prt', 'advmod']):
                        compound += ' ' + words[j]
                compound = compound[1:]
                ###=== get entity ===================================================================
                entity = ''
                for j in range(1, len(heads)):
                    if (j == entity_id):
                        entity += ' ' + words[j]
                    if (heads[j] == entity_id) and (labels[j] in ['compound', 'nummod']):
                        for k in range(1, len(heads)):
                            if (check_belong_to(j, k ,heads)):
                                entity += ' ' + words[k]
                entity = entity[1:]
                ###=== get direct object ===========================================================
                dobject = ''
                for j in range(1, len(heads)):
                    if (labels[j] == 'dobj') and (heads[j] == verb_id):
                        for k in range(1, len(heads)):
                            if (not check_belong_to(entity_id, k, heads)) and (check_belong_to(j, k, heads)):
                                dobject += ' ' + words[k]
                dobject = dobject[1:]
                ###=== generate question ===========================================================
                question = ''
                answer = ''
                if (ners[i] in ['LOCATION', 'ORGANIZATION', 'COUNTRY']):
                    if (first_aux == -1):
                        if (xpos[verb_id] == 'VBZ'):
                            question = 'where does' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject
                        if (xpos[verb_id] == 'VBP'):
                            question = 'where do' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject
                        if (xpos[verb_id] == 'VBD'):
                            question = 'where did' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject
                    else:
                        question = 'where ' + words[first_aux] + ' ' + subj + ' ' + verb + ' ' + dobject
                    tags.append(ners[i])

                if (ners[i] in ['DATE', 'TIME']):
                    if (first_aux == -1):
                        if (xpos[verb_id] == 'VBZ'):
                            question = 'when does' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject
                        if (xpos[verb_id] == 'VBP'):
                            question = 'when do' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject
                        if (xpos[verb_id] == 'VBD'):
                            question = 'when did' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject
                    else:
                        question = 'when ' + words[first_aux] + ' ' + subj + ' ' + verb + ' ' + dobject
                    tags.append(ners[i])
                answer = entity
                questions.append(question + '?')
                answers.append(answer)
                tags.append('nmod')
                list_tags.append(tags)
        """
            process sentences have form: subject + verb + xcomp/ccomp + in/on + TIME/LOCATION
        """  
        if (labels[i] in nmod_labels) and (labels[heads[i]] in ['xcomp', 'ccomp']) and (xpos[heads[heads[i]]] in verb_labels) and (ners[i] in ['TIME', 'DATE', 'LOCATION', 'ORGANIZATION', 'COUNTRY']):
            verb_id = heads[heads[i]]
            entity_id = i
            ###=== get subject_id ===============================================================
            subj_id = -1
            for j in range(1, len(heads)):
                if (heads[j] == verb_id) and (labels[j] in nsubj_labels):
                    subj_id = j
            if (subj_id != -1):
                ###=== exist auxulary or not ========================================================
                first_aux = -1
                for j in range(1, len(heads)):
                    if (heads[j] == verb_id) and (labels[j] in ['aux', 'auxpass']):
                        first_aux = j
                        break
                ###=== get subject ==================================================================
                subj = ''
                for j in range(1, len(heads)):
                    if (check_belong_to(subj_id, j, heads)):
                        subj += ' ' + words[j]
                subj = subj[1:]
                ###=== get verb =====================================================================
                illegal_id = []
                verb = ''
                for j in range(1, len(heads)):
                    if (heads[j] == verb_id) and (labels[j] not in ['compound', 'compound:prt', 'compound:svc', 'aux', 'auxpass']):
                        for k in range(1, len(heads)):
                            if (check_belong_to(j, k, heads)):
                                illegal_id.append(k)
                for j in range(1, len(heads)):
                    if (j != first_aux) and (j not in illegal_id) and (check_belong_to(verb_id, j, heads)):
                        verb += ' ' + words[j]
                verb = verb[1:]
                ###=== get verb compound=====================================================================
                compound = ''
                for j in range(1, len(heads)):
                    if (heads[j] == verb_id) and (labels[j] in ['compound:prt', 'advmod']):
                        compound += ' ' + words[j]
                compound = compound[1:]
                ###=== get entity ===================================================================
                illegal_id = []
                entity = ''
                for j in range(1, len(heads)):
                    if (j == entity_id):
                        entity += ' ' + words[j]
                        illegal_id.append(j)
                    if (heads[j] == entity_id) and (labels[j] in ['compound', 'nummod', 'det']):
                        for k in range(1, len(heads)):
                            if (check_belong_to(j, k ,heads)):
                                entity += ' ' + words[k]
                                illegal_id.append(k)
                entity = entity[1:]
                ###=== get direct object ===========================================================
                dobject = ''
                for j in range(1, len(heads)):
                    if (labels[j] == 'dobj') and (heads[j] == verb_id):
                        for k in range(1, len(heads)):
                            if (not check_belong_to(entity_id, k, heads)) and (check_belong_to(j, k, heads)):
                                dobject += ' ' + words[j]
                dobject = dobject[1:]
                ###=== get ccomp/xcomp part ========================================================
                extention = ''
                for j in range(1, len(heads)):
                    if (j not in illegal_id) and (not check_belong_to(entity_id, j, heads)) and (check_belong_to(heads[i], j, heads)):
                        extention += ' ' + words[j]
                ###=== generate question ===========================================================
                question = ''
                answer = ''
                if (ners[i] in ['LOCATION', 'ORGANIZATION', 'COUNTRY']):
                    if (first_aux == -1):
                        if (xpos[verb_id] == 'VBZ'):
                            question = 'where does' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject + ' ' + extention
                        if (xpos[verb_id] == 'VBP'):
                            question = 'where do' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject + ' ' + extention
                        if (xpos[verb_id] == 'VBD'):
                            question = 'where did' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject + ' ' + extention
                    else:
                        question = 'where ' + words[first_aux] + ' ' + subj + ' ' + verb + ' ' + dobject + ' ' + extention
                    tags.append(ners[i])

                if (ners[i] in ['DATE', 'TIME']):
                    if (first_aux == -1):
                        if (xpos[verb_id] == 'VBZ'):
                            question = 'when does' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject + ' ' + extention
                        if (xpos[verb_id] == 'VBP'):
                            question = 'when do' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject + ' ' + extention
                        if (xpos[verb_id] == 'VBD'):
                            question = 'when did' + ' ' + subj + ' ' + get_morphy(words[verb_id], xpos[verb_id]) + ' ' + compound + ' ' + dobject + ' ' + extention
                    else:
                        question = 'when ' + words[first_aux] + ' ' + subj + ' ' + verb + ' ' + dobject + ' ' + extention
                    tags.append(ners[i])
                answer = entity
                questions.append(question + '?')
                answers.append(answer)
                tags.append('nmod')
                list_tags.append(tags)
    return questions, answers, list_tags

In [16]:
"""
    nmod:tmod : A temporal modifier is a subtype of the nmod relation: if the modifier is specifying
     a time, it is labeled as tmod.
    Eg: last night, I swam in the pool
    question: when I swam in the pool?
    answer  : last night
"""

def generate_tmod_question(words, xpos, heads, labels, ners):
    questions = []
    answers = []
    relations = []

    ### find tmod relation
    for i in range(1, len(labels)):
        if (labels[i] in ['tmod', 'nmod:tmod', 'nmod:since']):
            head_id = heads[i]
            tail_id = i

            ### find tmod phrase (all words belong to tail-word)
            illegal_id = [] 
            tmod_phrase = ''
            for j in range(1, len(heads)):
                if (check_belong_to(tail_id, j, heads)):
                    tmod_phrase += ' ' + words[j]
                    illegal_id.append(j)
            tmod_phrase = tmod_phrase[1:]

            ### find body of question
            body_question = ''
            for j in range(1, len(heads)):
                if ((j not in illegal_id) and check_belong_to(head_id, j, heads)):
                    body_question += ' ' + words[j]
            body_question = body_question[1:]

            ### generate question
            question = 'when ' + body_question
            answer = tmod_phrase

            ### append
            questions.append(question)
            answers.append(answer)
            relations.append('tmod')

    return questions, answers, relations


In [17]:
"""
    if sentence in basic form: subject + verb + object -> form = 1 -> short question
    if sentence in ccomp form: subject + verb + clouse -> form = 2 -> long question
    if sentence in cop form  : subject + cop + noun/adj-> form = 3 
    ignore questions which answer's tag in 'PRP'
"""

def generate_subj_question(words, xpos, heads, labels, ners):
    questions = []
    answers = []
    list_tags = []
    min_qb_length = 5
    verb_labels = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    adj_labels = ['JJ', 'JJR', 'JJS']
    noun_labels = ['NN', 'NNP', 'NNS', 'NNPS', 'PRP']
    # find all word with nsubj label
    nsubj_labels = ['nsubj', 'nsubjpass']
    for i in range(1, len(labels)):
        if (labels[i] in nsubj_labels) and (xpos[i] not in ['PRP']) and (xpos[heads[i]] in verb_labels):
            tags = []
            is_human = False
            is_organization = False
            ### get all words belong to subject
            illegal_id = []
            for j in range(1, len(heads)):
                if ((labels[j] in ['ref', 'case', 'punct', 'appos']) and (heads[j] == i)):
                    for k in range(1, len(heads)):
                        if (check_belong_to(j, k, heads)):
                            illegal_id.append(k)
            answer = ''
            for j in range(1, len(heads)):
                if (check_belong_to(i, j, heads)):
                    if (j not in illegal_id):
                        answer += ' ' + words[j]
                    last_answer_id = j
            answer = answer[1:]
            # check noun info
            is_human, _ = get_noun_info(words[i], xpos[i])
            if (ners[i] == 'PERSON'):
                is_human = True
            if (ners[i] == 'ORGANIZATION'):
                is_organization = True
        
            ### get all words belong to predicate
            # get form of sentence
            form = 0
            for j in range(1, len(heads)):
                if (heads[j] == heads[i]):
                    if (labels[j] == 'dobj'):
                        form = 1
                    if (labels[j] in ['ccomp', 'xcomp']) or ((len(labels[j]) >= 5) and (labels[j][:5] == 'advcl')):
                        form = 2
            # get question in form 1
            question_body = ''
            if form == 1:
                illegal_id = []
                for j in range(last_answer_id + 1, len(heads)):
                    if (heads[j] == heads[i]) and not (labels[j] in ['dobj', 'aux', 'auxpass', 'cop', 'advmod', 'ref', 'compound', 'neg']):
                        for k in range(1, len(heads)):
                            if check_belong_to(j, k, heads):
                                illegal_id.append(k)
                for j in range(last_answer_id + 1, len(heads)):
                    if (j not in illegal_id) and (check_belong_to(heads[i], j, heads)):
                        if (words[j] == 'am'):
                            question_body += ' is'
                        else:
                            question_body += ' ' + words[j]
            # change to form 2 if length question too short
            if (form == 1) and (len(question_body.split(" ")) < min_qb_length):
                question_body = ""
                form = 2
            # get question in form 2
            if form == 2:
                illegal_id = []
                for j in range(last_answer_id + 1, len(heads)):
                    if words[j] == ".":
                        illegal_id.append(j)
                for j in range(last_answer_id + 1, len(heads)):
                    if (j not in illegal_id) and (check_belong_to(heads[i], j, heads)):
                        if (words[j] == 'am'):
                            question_body += ' is'
                        else:
                            question_body += ' ' + words[j]
            if (question_body == ''):
                continue
            if is_human:
                question = 'who' + question_body
                tags.append('PERSON')
            elif is_organization:
                question = 'which organization' + question_body
                tags.append('ORGANIZATION')
            else:
                question = 'what' + question_body

            # append
            questions.append(question + '?')
            answers.append(answer)
            tags.append('subj')
            list_tags.append(tags)
        if (labels[i] in nsubj_labels) and (xpos[i] not in ['PRP']) and (xpos[heads[i]] in adj_labels + noun_labels):
            tags = []
            is_human = False
            is_organization = False
            # check if in cop form
            has_cop = False
            for j in range(1, len(heads)):
                if (heads[j] == heads[i]) and (labels[j] == 'cop'):
                    has_cop = True
            if not has_cop:
                continue
            ### get all words belong to subject
            illegal_id = []
            for j in range(1, len(heads)):
                if ((labels[j] in ['ref', 'case', 'punct', 'appos']) and (heads[j] == i)):
                    for k in range(1, len(heads)):
                        if (check_belong_to(j, k, heads)):
                            illegal_id.append(k)
            answer = ''
            for j in range(1, len(heads)):
                if (check_belong_to(i, j, heads)):
                    if (j not in illegal_id):
                        answer += ' ' + words[j]
                    last_answer_id = j
            answer = answer[1:]
            # check noun info
            is_human, _ = get_noun_info(words[i], xpos[i])
            if (ners[i] == 'PERSON'):
                is_human = True
            if (ners[i] == 'ORGANIZATION'):
                is_organization = True
            # get question body
            question_body = ""
            for j in range(1, len(heads)):
                if (j == heads[i]):
                    question_body += ' ' + words[j]
                if (heads[j] == heads[i]) and ((labels[j] in ['cop', 'det', 'advmod', 'aux', 'auxpass', 'neg', 'case', 'amod']) or ((len(labels[j]) >= 4) and (labels[j][:4] == 'nmod'))):
                    for k in range(1, len(heads)):
                        if (check_belong_to(j, k, heads)):
                            if (words[k] == 'am'):
                                question_body += ' is'
                            else:
                                question_body += ' ' + words[k]
            if (question_body == ''):
                continue
            if is_human:
                question = 'who' + question_body
                tags.append('PERSON')
            elif is_organization:
                question = 'which organization' + question_body
                tags.append('ORGANIZATION')
            else:
                question = 'what' + question_body
            # append
            questions.append(question + '?')
            answers.append(answer)
            tags.append('subj')
            list_tags.append(tags)
    return questions, answers, list_tags

In [18]:
"""
    ccomp: clausal complement
    Verb -- ccomp -- Verb : 68%
    Verb -- ccomp -- Adj  : 10%
    Verb -- ccomp -- Noun : 9%
    Eg: He says that you like to swim
    => question: what he say? = what + nsubj (subject of verb) + verb
    => answer  : you like swim = clausal complement
"""

def generate_ccomp_xcomp_question(words, xpos, heads, labels, ners):
    ###
    questions = []
    answers = []
    list_tags = []
    verb_labels = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    adj_labels = ['JJ', 'JJR', 'JJS']
    noun_labels = ['NN', 'NNP', 'NNS', 'NNPS']
    nsubj_labels = ['nsubj', 'nsubjpass']

    ### find all ccomp relations
    for i in range(1, len(labels)):
        if (labels[i] in ['ccomp', 'xcomp']):
            head_id = heads[i]
            tail_id = i
            tags = []
            ### VERB ----ccomp----> VERB/ADJ/NOUN
            if ((xpos[head_id] in verb_labels) and (xpos[tail_id] in (verb_labels + adj_labels + noun_labels))):
                ### --Nsubject--> VERB --ccomp--> VERB
                # get nsubj of sentence
                nsubj = ''
                for j in range(1, len(labels)):
                    if ((heads[j] == head_id) and (labels[j] in nsubj_labels)):
                        nsubj = ''
                        illegal_id = []
                        if xpos[j] in ['NNP', 'NNPS']:
                            for k1 in range(1, len(heads)):
                                if (heads[k1] == j) and ((labels[k1] in ['appos', 'punct']) or ((len(labels[k1]) > 4) and (labels[k1][:4] == 'nmod'))):
                                    for k2 in range(1, len(heads)):
                                        if check_belong_to(k1, k2, heads):
                                            illegal_id.append(k2)
                        for k in range(1, len(labels)):
                            if (k not in illegal_id) and (check_belong_to(j, k, heads)):
                                nsubj += ' ' + words[k]
                        nsubj = nsubj[1:]
                # can not generate question without subject
                if (nsubj == ''):
                    continue
                # get object of sentence
                dobj = ''
                for j in range(1, len(heads)):
                    if (heads[j] == head_id) and (labels[j] in ['dobj', 'iobj']):
                        dobj = ''
                        for k in range(1, len(labels)):
                            if (check_belong_to(j, k, heads)):
                                dobj += ' ' + words[k]
                dobj = dobj[1:]
                # get auxiliary of verb
                aux = ''
                for j in range(1, len(heads)):
                    if (heads[j] == head_id) and (labels[j] == 'aux'):
                        aux = ''
                        for k in range(1, len(labels)):
                            if (check_belong_to(j, k, heads)):
                                aux += ' ' + words[k]
                aux = aux[1:]
                # get compound of verb
                compound = ''
                for j in range(1, len(heads)):
                    if (heads[j] == head_id) and (labels[j] in ['compound:prt']):
                        compound += ' ' + words[j]
                compound = compound[1:] 
                # get nmod of verb
                nmod = ''
                for j in range(1, len(heads)):
                    if (heads[j] == head_id) and (len(labels[j]) >= 4) and (labels[j][:4] == 'nmod'):
                        for k in range(1, len(labels)):
                            if (check_belong_to(j, k, heads)):
                                nmod += ' ' + words[k]
                nmod = nmod[1:]
                # get all clausal complement except 'mark' relation
                verb_clausal = ''
                illegal_id = []
                for j in range(1, len(labels)):
                    if ((heads[j] == i) and (labels[j] in ['mark'])):
                        for k in range(1, len(labels)):
                            if (check_belong_to(j, k, heads)):
                                illegal_id.append(k)

                for j in range(1, len(labels)):
                    if (j not in illegal_id) and (check_belong_to(i, j, heads)):
                        verb_clausal += ' ' + words[j]
                verb_clausal = verb_clausal[1:]
                # generate question + process verb form
                if aux == '':
                    if xpos[head_id] == 'VBD':
                        question = 'what did ' + nsubj + ' ' + get_morphy(words[head_id], 'VBD')
                    elif (xpos[head_id] == 'VBZ') and (wn.morphy(words[head_id], 'v') != 'be'):
                        question = 'what does ' + nsubj + ' ' + get_morphy(words[head_id], 'VBZ')
                    elif (xpos[head_id] == 'VBZ') and (wn.morphy(words[head_id], 'v') == 'be'):
                        question = 'what ' + words[head_id] + ' ' + nsubj
                elif len(aux.split(' ')) == 1:
                    question = 'what ' + aux + ' ' + nsubj + ' ' + words[head_id] + ' ' + compound
                else:
                    question = 'what ' + nsubj + ' ' + words[head_id] + ' ' + compound
                if dobj != '':
                    question += ' ' + dobj
                if (nmod != '') and (compound == ''):
                    question += ' ' + nmod
                answer = verb_clausal

                # append
                questions.append(question + '?')
                answers.append(answer)
                tags.append('ccomp')
                list_tags.append(tags)
            break
    ### return
    return questions, answers, list_tags

In [25]:
sentence = "I sleep at 2 pm"
# process nlp tasks
raw_pos = pos_tagger.transform(sentence)
raw_dp = dependparser.predict({'text' : sentence})
raw_ner = ner.transform(sentence)
words, xpos = get_pos_conll(raw_pos)
ners = get_ner_conll(raw_ner)
dependency_trees = get_dp_conll(raw_dp, words, xpos, ners)
# add to sentences data
sentence_data = dependency_trees[0]

In [27]:
print(sentence_data)

{'xpos': ['', 'PRP', 'VBP', 'IN', 'CD', 'NN'], 'ners': ['', None, None, None, 'NUMBER', None], 'heads': [-1, 2, 0, 5, 5, 2], 'words': ['', 'I', 'sleep', 'at', '2', 'pm'], 'labels': ['', 'nsubj', 'root', 'case', 'nummod', 'nmod:at']}


In [28]:
words = sentence_data['words']
xpos = sentence_data['xpos']
heads = sentence_data['heads']
labels = sentence_data['labels']
ners = sentence_data['ners']
questions, answers, relations = generate_ccomp_xcomp_question(words, xpos, heads, labels, ners)

In [29]:
for q, a in zip(questions, answers):
    print(q)
    print(a)